In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.stats import truncnorm
from numba import jit, njit
from shapely import MultiPoint, Point
from shapely.ops import nearest_points

Read dwelling types

In [2]:
dwellingTypes = pd.read_csv('./Data/DwellingTypesByZone2006.csv')

Read Zonal Profile data

In [3]:
profileData = pd.read_csv('./Data/GTAProfileData.csv')

Create ZoneSystem file which contains the zones to be used in the sim

In [4]:
zoneSystem = pd.DataFrame(data=profileData['DAUID'], columns=['DAUID'], dtype=str)
zoneSystem = zoneSystem[zoneSystem['DAUID'].str.len()==8]

Read Shapefile Zones

In [5]:
zoneShape:gpd.GeoDataFrame = gpd.read_file('./Data/TorontoZoneShape/TorontoZones_26917.shp')
zoneShape = zoneShape[zoneShape['DAUID'].astype(str).isin(zoneSystem['DAUID'])]

Create Zone Dataframe that holds all data

zoneData = pd.DataFrame(columns=['DAUID', 'Population', 'Employment rate', 'Area', "Total - Structural type of dwelling"])

for i, zone in zoneSystem.iterrows():
    numDwellings=200
    population = 400
    try:numDwellings = dwellingTypes.loc[dwellingTypes['Geography'].astype(str)==zone['DAUID'], 'Total - Structural type of dwelling'].iloc[0]
    except:pass
    try:population = int(profileData.loc[profileData['DAUID'].astype(str)==zone['DAUID'], "Population"].iloc[0])
    except:pass

    zoneData.loc[len(zoneData)] = [zone['DAUID'], population, profileData.loc[profileData['DAUID'].astype(str)==zone['DAUID'], "Employment rate"].iloc[0], float(zoneShape.loc[zoneShape['DAUID'].astype(str)==zone['DAUID'], 'geometry'].iloc[0].area), numDwellings]

zoneData.to_csv('./Data/combinedZoneData.csv', index=False)

<span style="font-size:5em;">CREATE PERSONS</span>

In [6]:
zoneData = pd.read_csv('./Data/combinedZoneData.csv')

Income generator

In [7]:
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)


In [8]:
salary_distribution_employed = get_truncated_normal(mean=70000, sd=40000, low=30000, upp=200000)
salary_distribution_retired = get_truncated_normal(mean=40000, sd=20000, low=15000, upp=70000)

In [9]:
totalPersons = zoneData['Population'].astype(float).sum()

In [10]:
employment_incomes = salary_distribution_employed.rvs(int(totalPersons*.8))
retired_incomes = salary_distribution_retired.rvs(int(totalPersons*.4))

<h3>synthesize persons<h3/>

initial person counts per zone

In [11]:
personZoneCounts = np.empty([0,2], dtype=np.float64)
for i, zone in zoneData.iterrows():
    personZoneCounts = np.r_[personZoneCounts, [[np.float64(zone['DAUID']), np.float64(zone['Population'])]]]

In [12]:
personZoneCounts

array([[3.5190002e+07, 7.9000000e+02],
       [3.5190003e+07, 3.8400000e+02],
       [3.5190004e+07, 6.8700000e+02],
       ...,
       [3.5250973e+07, 4.9900000e+02],
       [3.5250974e+07, 4.7700000e+02],
       [3.5250975e+07, 4.7600000e+02]])

Create numba function to synthesize persons

In [13]:
@jit(nopython=True)
def createPersons(personCounts):
    p = np.empty((0,10))
    for zoneNum in range(personCounts.shape[0]):
        print(zoneNum)
        numPeopleInZone = personCounts[zoneNum][1]
        for person in range(numPeopleInZone):
            age = np.random.randint(0,100)
            gender = np.random.randint(0,2)
            # 0:SINGLE 1:MARRIED 2:CHILD
            relationship = 2 if age < 18 else np.random.randint(0, 2)
            occupation = 0 if age < 4 \
                    else 4 if age > 65 \
                    else 3 if age < 22 \
                    else 1 if np.random.random() < .70 \
                    else 2
            occupation_type = 1
            workplace = -1
            income=-1
            if occupation==1:
                income = employment_incomes[np.random.randint(0,int(totalPersons*.8))]
            elif occupation==4:
                income = retired_incomes[np.random.randint(0,int(totalPersons*.4))]
            
            schoolplace=-1
            
            p = np.append(p, np.array([[int(personZoneCounts[zoneNum][0]), -1, age, gender, relationship, occupation, occupation_type, workplace, income, schoolplace]]), axis=0)
    return p
    

Create dataframe of persons using the c-compiled function above to create the data

In [14]:
persons = pd.DataFrame(data=createPersons(personZoneCounts), columns=['id', 'hhId', 'age', 'gender', 'relationship', 'occupation', 'occupation_type', 'workplace', 'income', 'schoolplace'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145


Clean up varaibles

In [ ]:
# del employment_incomes
# del retired_incomes
# del personZoneCounts
# del profileData
# del dwellingTypes